In [1]:
import os

from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import VecMonitor
from stable_baselines3 import PPO
from sumo_rl import parallel_env
import supersuit as ss

from tyre_pm_reward import tyre_pm_reward

c:\Users\Justin Mak\anaconda3\envs\rl\lib\site-packages\gymnasium\envs\registration.py:693: UserWarning: WARN: Overriding environment GymV26Environment-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [2]:
# Create environment
env = parallel_env(
    net_file=os.path.join("nets","RESCO","grid4x4","grid4x4.net.xml"),
    route_file=os.path.join("nets","RESCO","grid4x4","grid4x4_1.rou.xml"),
    num_seconds=3600,
    out_csv_name=os.path.join("outputs","grid4x4","ppo"),
    reward_fn=tyre_pm_reward
)
env = ss.pettingzoo_env_to_vec_env_v1(env)
env = ss.concat_vec_envs_v1(env, 2, num_cpus=1, base_class="stable_baselines3")
env = VecMonitor(env)

In [3]:
model = PPO(
    "MlpPolicy",
    env,
    learning_rate=5e-4,
    n_steps=256,
    batch_size=256,
    n_epochs=5,
    gamma=0.95,
    gae_lambda=0.99,
    clip_range=0.3,
    ent_coef=0.09,
    vf_coef=0.05,
    max_grad_norm=0.9,
    tensorboard_log=os.path.join("logs","grid4x4","ppo_test"),
    verbose=1,
)

Using cuda device


In [4]:
# Evaluate untrained random agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=1)

print(mean_reward)
print(std_reward)

KeyboardInterrupt: 

In [ ]:
model.learn(total_timesteps=5e4)

In [ ]:
# Evaluate trained agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=1)

print(mean_reward)
print(std_reward)